In [1]:
from mesa.experimental import JupyterViz

from VanetTraceLoader import get_traces, VehicleTrace
from render import make_render_station_load_chart, make_render_station_vehicle_count_chart, render_model_with_bg, \
    render_model_orientations, render_vehicle_loads
from capacity_vanet import VECModel, get_grid, SEED, DefaultOffloadingStrategy, NearestRSUStrategy, \
    CRETEIL_4_RSU_HALF_CAPA_CONFIG, \
    StaticVehicleLoadGenerator, BEST_DEFAULT_CONFIG, DynamicVehicleLoadGenerator

In [2]:
page = JupyterViz(
    VECModel,
    model_params={"rs_strategy": DefaultOffloadingStrategy(**BEST_DEFAULT_CONFIG),
                  "rsu_configs": CRETEIL_4_RSU_HALF_CAPA_CONFIG,
                  "vehicle_load_gen": DynamicVehicleLoadGenerator(), "traces": get_traces(morning=True, eval=True),
                  "load_update_interval": 1, "start_at": 0},
    measures=[
        render_model_with_bg(get_grid()),
        render_model_orientations,
        render_vehicle_loads,
        make_render_station_load_chart(tail=100),
        make_render_station_vehicle_count_chart(tail=100),
        "TotalSuccessfulHandoverCount",
        "TotalFailedHandoverCount",
        "AvgQoS",
        "MinQoS",
        "GiniLoad",
        "VehicleCount",
    ],
    name="VEC Model",
    space_drawer=False,
    seed=SEED,
)

# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text